In [1]:
###############
##Version 0:IDW on NDBC 
###############

In [2]:
import os
import numpy as np
import six.moves.cPickle as pickle
import torch
import torch.nn as nn
import torch.utils.data as data
import os
import pandas as pd
import torch

# Correct file path
# file_path = '/home/xren451/rxb/phd/Spatial_interpolation/XBSPA/ModIGNNK/data/NDBC/all.npy'
file_path = 'data/NDBC/all.npy'
station_value = np.load(file_path)
station_value=station_value.transpose(2,0,1)
lat_file_path = 'data/NDBC/Station_info_edit.csv'
station_info=np.array(pd.read_csv(lat_file_path,header=None))
station_value = station_value[:, :, 5:13]
station_value=torch.tensor(station_value)

In [10]:
lat_file_path = 'data/NDBC/Station_info_edit.csv'
station_info=pd.read_csv(lat_file_path,header=None)
station_info.head()

,0,1,2
0,41001,34.703,72.242
1,41008,31.400,80.866
2,41012,30.042,80.534
3,41013,33.441,77.764
4,41024,33.837,78.477


In [17]:
import numpy as np
import torch
import pandas as pd
from scipy.spatial import distance_matrix
import random

def idw_interpolation(target_coord, known_coords, known_values, power=2):
    distances = distance_matrix([target_coord], known_coords)[0]
    # Handle case where distance is zero (i.e., target_coord is exactly at a known_coord)
    if np.any(distances == 0):
        return known_values[np.argmin(distances)]
    weights = 1 / (distances ** power)
    weights /= weights.sum()  # Normalize weights
    interpolated_value = np.dot(weights, known_values)
    return interpolated_value

def perform_idw_for_feature(station_info, station_values, feature_index, target_coords):
    feature_values = station_values[:, :, feature_index]  # Extract specific feature values
    interpolated_values = []
    
    for t in range(feature_values.shape[1]):  # Iterate over timesteps
        known_indices = ~torch.isnan(feature_values[:, t])  # Identify stations with known values
        if known_indices.sum() == 0:
            interpolated_values.append(np.nan)  # If no known values, return NaN
            continue
        
        known_coords = station_info[known_indices.numpy(), 1:3]  # Coordinates of known stations (2D)
        known_values = feature_values[known_indices, t].numpy()  # Known feature values
        
        interpolated_value = idw_interpolation(target_coords, known_coords, known_values)
        interpolated_values.append(interpolated_value)
    
    return np.array(interpolated_values)

def compute_mse(true_values, predicted_values):
    return np.nanmean((true_values - predicted_values) ** 2)

# Example usage
lat_file_path = 'data/NDBC/Station_info_edit.csv'
station_info = pd.read_csv(lat_file_path, header=None).values  # Convert to NumPy array directly
station_values_tensor = torch.tensor(station_value)  # Assuming station_values is your tensor

feature_index = 0   # Specify the feature index you are interested in

# Randomly select a target station
target_index = random.choice(range(station_info.shape[0]))
target_coords = station_info[target_index, 1:3]  # Assuming the coordinates are in columns 1 and 2

# Get the true values for the selected station
true_values = station_values_tensor[target_index, :, feature_index].numpy()

# Perform IDW interpolation
interpolated_results = perform_idw_for_feature(station_info, station_values_tensor, feature_index, target_coords)

# Compute the MSE
mse = compute_mse(true_values, interpolated_results)
print(f"Target station index: {target_index}")
print(f"True values: {true_values}")
print(f"Interpolated values: {interpolated_results}")
print(f"MSE: {mse}")


/tmp/ipykernel_1020502/2591001307.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  station_values_tensor = torch.tensor(station_value)  # Assuming station_values is your tensor


Target station index: 44
True values: [205.53244 205.53244 205.53244 ... 287.      290.      282.     ]
Interpolated values: [205.53244 205.53244 205.53244 ... 287.      290.      282.     ]
MSE: 0.0


In [ ]:
import numpy as np
import pandas as pd
import random
import time
import tensorflow as tf
from scipy.spatial import ConvexHull
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr
from shapely.geometry import Point, Polygon
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from spektral.layers import GCNConv

# Load datasets
station_value_path = 'data/NDBC/all.npy'
station_info_path = 'data/NDBC/Station_info_edit.csv'

station_value = np.load(station_value_path)
station_value = station_value.transpose(2, 0, 1)[:, :, 5:13]  # Select relevant features
station_info = pd.read_csv(station_info_path, header=None).values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
station_value = station_value.reshape(-1, station_value.shape[-1])
station_value = scaler.fit_transform(station_value)
station_value = station_value.reshape(103, 8784, 8)

# Utility functions
def idw_interpolation(station_coords, temperatures, target_point, power=2):
    distances = np.linalg.norm(station_coords - target_point, axis=1)
    weights = 1 / (distances ** power)
    weights /= weights.sum()
    interpolated_value = np.dot(weights, temperatures)
    return interpolated_value

def calculate_inverse_distance_adj(station_coords):
    num_nodes = station_coords.shape[0]
    adj = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.linalg.norm(station_coords[i] - station_coords[j])
                adj[i, j] = np.exp(-distance)
    np.fill_diagonal(adj, 1)  # Add self-loops
    return adj

def GCN_model(input_shape, adj_shape):
    inputs_feat = Input(shape=(input_shape,))
    inputs_adj = Input(shape=(adj_shape[0], adj_shape[1]), sparse=True)
    gcn_output = GCNConv(16, activation='relu')([inputs_feat, inputs_adj])
    model = Model(inputs=[inputs_feat, inputs_adj], outputs=gcn_output)
    return model

# Convert station_info to a dictionary for easy lookup
station_coords_dict = {row[0]: (float(row[1]), float(row[2])) for row in station_info}
station_coords = np.array(list(station_coords_dict.values()))  # Convert to numpy array for vectorized operations

# Prepare data for GCN
adj_matrix = calculate_inverse_distance_adj(station_info[:, 1:3].astype(float))
adj_matrix_sparse = tf.sparse.from_dense(adj_matrix)
x = station_value[:, :, target_feature].transpose(1, 0)  # (time_steps, num_stations, 1)

# Ensure the input to the GCN is 2D
x = x.reshape(x.shape[0], x.shape[1], 1)

# Define and train the GCN model
gcn_model = GCN_model(x.shape[1], adj_matrix.shape)
optimizer = Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.MeanSquaredError()

# Training loop
epochs = 100
batch_size = 10
start_time = time.time()

for epoch in range(epochs):
    epoch_loss = 0
    num_batches = x.shape[0] // batch_size
    for batch in range(num_batches):
        with tf.GradientTape() as tape:
            batch_indices = range(batch * batch_size, (batch + 1) * batch_size)
            gcn_outputs = []
            for t in batch_indices:
                gcn_output = gcn_model([x[t], adj_matrix_sparse])
                gcn_outputs.append(gcn_output)

            H_gcn = tf.stack(gcn_outputs, axis=0)
            H_gcn = tf.reshape(H_gcn, (batch_size, station_info.shape[0], -1))  # (batch_size, num_stations, 16)

            # MoE aggregation
            H_pos_all = np.random.rand(station_info.shape[0], 16)  # Dummy positional encoding for stations
            H_pos_one = np.random.rand(16)  # Dummy positional encoding for target station
            estimated_values = tf.einsum('bnd,nd->bn', H_gcn, H_pos_all)  # (batch_size, num_stations)
            estimated_values = tf.einsum('bn,d->b', estimated_values, H_pos_one)  # (batch_size,)

            batch_true_values = idw_estimate[batch_indices]
            loss = loss_fn(batch_true_values, estimated_values)

        grads = tape.gradient(loss, gcn_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, gcn_model.trainable_variables))
        epoch_loss += loss.numpy()

    epoch_loss /= num_batches
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss}')

training_time = time.time() - start_time

# Predict final values
gcn_outputs = []
for t in range(x.shape[0]):
    gcn_output = gcn_model([x[t], adj_matrix_sparse])
    gcn_outputs.append(gcn_output)

H_gcn = tf.stack(gcn_outputs, axis=0)
H_gcn = tf.reshape(H_gcn, (x.shape[0], station_info.shape[0], -1))  # (time_steps, num_stations, 16)

# MoE aggregation for predictions
H_pos_all = np.random.rand(station_info.shape[0], 16)  # Dummy positional encoding for stations
H_pos_one = np.random.rand(16)  # Dummy positional encoding for target station
estimated_values = tf.einsum('tnd,nd->tn', H_gcn, H_pos_all)  # (time_steps, num_stations)
estimated_values = tf.einsum('tn,d->t', estimated_values, H_pos_one)  # (time_steps,)

# Convert Tensor to NumPy array before reshaping
estimated_values = estimated_values.numpy()

# Denormalize the predictions
estimated_values = scaler.inverse_transform(estimated_values.reshape(-1, 1)).flatten()
idw_estimate = scaler.inverse_transform(idw_estimate.reshape(-1, 1)).flatten()

# Compute MAE and RMSE
mae = mean_absolute_error(idw_estimate, estimated_values)
rmse = mean_squared_error(idw_estimate, estimated_values, squared=False)
print(f'MAE: {mae}, RMSE: {rmse}, Training Time: {training_time} seconds')
